# **Genetic Algorithm - Guessing a Sentence**

For a sentence of length $L$ constructed from $N_C$ possible characters, the number of permutations of possible sentences is given by $(N_C)^L$.

If multiple sentences are constructed each generation, it would take $\frac{(N_C)^L}{N_P}$ generations on average to generate a specific sentence.
This number grows large quickly, and so this genetic algorithm aims to implement a more efficient algorithm to find a solution.

In [1]:
from __future__ import annotations

from typing import List

import numpy as np

from src.ga import GeneticAlgorithm
from src.helpers import print_system_msg
from src.member import Member
from src.population import Population

In [2]:
class PhraseSolverMember(Member):
    """
    Member to use in PhraseSolver app.
    """

    def __init__(self, length: int, gene_pool: List[str]) -> None:
        """
        Initialise PhraseSolverMember with length of phrase and gene pool.

        Parameters:
            length (int): Length of member chromosome
            gene_pool (List[str]): List of possible characters
        """
        super().__init__()
        self._length = length
        self._gene_pool = gene_pool
        self._chromosome = "".join([self.random_char for _ in range(self._length)])

    @property
    def random_char(self) -> str:
        """
        Return a random gene from the possible genes.
        """
        _choice: str = np.random.choice(self._gene_pool)
        return _choice

    @property
    def fitness(self) -> int:
        """
        Return member fitness.
        """
        return self._score**2

    def calculate_score(self, phrase: str) -> None:
        """
        Calculate the member's score based on the provided phrase.

        Parameters:
            phrase (str): Used to compare chromosome to phrase and calculate fitness
        """
        self._score = sum([self._chromosome[i] == phrase[i] for i in range(self._length)])

    def crossover(self, parent_a: Member, parent_b: Member, mutation_rate: int) -> None:
        """
        Crossover the chromosomes of two parents to create a new chromosome.

        Parameters:
            parent_a (Member): Used to construct new chromosome
            parent_b (Member): Used to construct new chromosome
            mutation_rate (int): Probability for mutations to occur
        """
        self._new_chromosome = ""

        for i in range(self._length):
            prob = np.random.randint(0, 100)

            # Half of the genes will come from parentA
            if prob < (100 - mutation_rate) / 2:
                new_char = parent_a._chromosome[i]
            # Half of the genes will come from parentB
            elif prob < (100 - mutation_rate):
                new_char = parent_b._chromosome[i]
            # Chance for a random genes to be selected
            else:
                new_char = self.random_char

            self._new_chromosome += new_char

In [3]:
class PhraseSolver(GeneticAlgorithm):
    """
    Simple app to use genetic algorithms to solve an alphanumeric phrase.
    """

    def __init__(self, mutation_rate: int) -> None:
        """
        Initialise PhraseSolver app.

        Parameters:
            mutation_rate (int)
        """
        super().__init__(mutation_rate)
        self._phrase: str

    def __str__(self) -> str:
        _phrase_str = f"Population Phrase: {self._phrase}"
        return f"{super().__str__()}\n{_phrase_str}"

    @classmethod
    def create_and_run(
        cls, population_size: int, mutation_rate: int, phrase: str, mem_genes: List[str]
    ) -> PhraseSolver:
        """
        Create app and run genetic algorithm.

        Parameters:
            population_size (int): Number of members in population
            mutation_rate (int): Mutation rate for members
            phrase (str): Phrase for members to solve
            mem_genes (List[str]): List of possible member genes

        Returns:
            ga (PhraseSolver): Phrase solver app
        """
        ga = cls(mutation_rate)
        ga._phrase = phrase
        ga._population = Population([PhraseSolverMember(len(ga._phrase), mem_genes) for _ in range(population_size)])
        print(ga)
        ga.run()
        return ga

    def _evaluate(self) -> None:
        """
        Evaluate the population.
        """
        self._population.calculate_member_scores(self._phrase)
        self._population.evaluate()

    def _analyse(self) -> None:
        """
        Analyse best member's chromosome.
        """
        _gen_text = f"Generation {self._generation:>4}:"

        # Correct phrase found so break out of the loop
        if self._population.best_chromosome == self._phrase:
            print_system_msg(f"{_gen_text} {self._population.best_chromosome} \t|| Solved!")
            self._running = False
            return

        # Return the closest match and its associated fitness then evolve.
        print_system_msg(
            f"{_gen_text} {self._population.best_chromosome} \t|| Max Fitness: {self._population.best_fitness}"
        )

In [4]:
population_size = 200
mutation_rate = 3
phrase = "I am a genetic algorithm!"
gene_pool = list("0123456789 abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ.,!?")

In [7]:
ga = PhraseSolver.create_and_run(population_size, mutation_rate, phrase, gene_pool)

[15-04-2024 | 01:00:49] Creating population...
Population Size: 200
Mutation Rate: 3
Population Phrase: I am a genetic algorithm!
[15-04-2024 | 01:00:49] Running algorithm...
[15-04-2024 | 01:00:49] Generation    1: s Cwpn SG?rfmt7v3oJSeOCF! 	|| Max Fitness: 9
[15-04-2024 | 01:00:49] Generation    2: 8kCi F SGOufSt a3oJmaOCF! 	|| Max Fitness: 25
[15-04-2024 | 01:00:49] Generation    3: b6uG cZL6OuvvU 3Fgcmaihm! 	|| Max Fitness: 36
[15-04-2024 | 01:00:49] Generation    4: s aoEI .zG !rc vlANRi5vj! 	|| Max Fitness: 64
[15-04-2024 | 01:00:49] Generation    5: I aM . SzO !Sc alANma5vF! 	|| Max Fitness: 100
[15-04-2024 | 01:00:49] Generation    6: I ai . hZn !mc alrNma5hC! 	|| Max Fitness: 144
[15-04-2024 | 01:00:49] Generation    7: I aN I JZnefTc 8lA5oILhj! 	|| Max Fitness: 144
[15-04-2024 | 01:00:49] Generation    8: I ai . hZn !mc JlqNm!5h0! 	|| Max Fitness: 121
[15-04-2024 | 01:00:49] Generation    9: 8 aN a LHDr!St algXksLhm! 	|| Max Fitness: 144
[15-04-2024 | 01:00:49] Generation   1